In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Capstone_Project"
!ls

/content/drive/MyDrive/Capstone_Project
annotations
checkpoints
COCO_train2014_000000581909.jpg
error
image_captioning_3_model_train_test_mscoco2014_10k_data.ipynb
image_captioning_3_model_train_test_mscoco2014_1k_data.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_100_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_150_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_300_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_50_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data.ipynb
image_captioning_3_model_train_test_mscoco2014_50k_data.ipynb
image_captioning_load_model_text_similarity_v1.ipynb
image_captioning_load_model_text_similarity_v2.ipynb
materials
old
output_10k
output_1k
output_20k
output_20k_100_epoch
output_20k_150_epoch
output_20k_20_epoch
output_20k_300_epoch
output_20k_50_epoch
output_50k
output_nlp
sentence_similarity_custom_model_A_v1.ipynb
sentence_similarity_custom_model_B_v1.ipynb

In [ ]:
!pip install -q sentence_transformers
!pip install -q huggingface-hub
!pip install datasets

     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 4.0 MB 68.3 MB/s 
     |████████████████████████████████| 1.2 MB 94.3 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 596 kB 57.1 MB/s 
     |████████████████████████████████| 6.5 MB 59.5 MB/s 
     |████████████████████████████████| 895 kB 74.0 MB/s 
     |████████████████████████████████| 325 kB 14.7 MB/s 
     |████████████████████████████████| 1.1 MB 89.0 MB/s 
     |████████████████████████████████| 136 kB 101.6 MB/s 
     |████████████████████████████████| 212 kB 78.5 MB/s 
     |████████████████████████████████| 127 kB 74.9 MB/s 
     |████████████████████████████████| 144 kB 88.3 MB/s 
     |████████████████████████████████| 271 kB 92.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uni

In [ ]:
from datasets import load_dataset

snli = load_dataset("snli", split="train")
dataset = snli
snli

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 550152
})

In [ ]:
print(snli[0])
print(snli[1])

{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}
{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is at a diner, ordering an omelette.', 'label': 2}


In [ ]:
print(f"Number of entries in dataset BEFORE removal of invalid entries: {len(dataset)}")
dataset = dataset.filter(
    lambda x: False if x['label'] == -1 else True
)
print(f"Number of entries in dataset AFTER removal of invalid entries: {len(dataset)}")

Number of entries in dataset BEFORE removal of invalid entries: 550152


  0%|          | 0/551 [00:00<?, ?ba/s]

Number of entries in dataset AFTER removal of invalid entries: 549367


In [ ]:
import torch
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# start from a pretrained bert-base-uncased model
model = BertModel.from_pretrained('bert-base-uncased')
# model

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
all_cols = ['label']

for part in ['premise', 'hypothesis']:
    dataset = dataset.map(
        lambda x: tokenizer(
            x[part], max_length=128, padding='max_length',
            truncation=True
        ), batched=True
    )
    for col in ['input_ids', 'attention_mask']:
        dataset = dataset.rename_column(
            col, part+'_'+col
        )
        all_cols.append(part+'_'+col)
print(all_cols)

  0%|          | 0/550 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

['label', 'premise_input_ids', 'premise_attention_mask', 'hypothesis_input_ids', 'hypothesis_attention_mask']


In [ ]:
# covert dataset features to PyTorch tensors
dataset.set_format(type='torch', columns=all_cols)

# initialize the dataloader
batch_size = 16
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# define mean pooling function
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(
        token_embeds.size()
    ).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool

In [ ]:
dataset[0].keys()

dict_keys(['label', 'premise_input_ids', 'premise_attention_mask', 'hypothesis_input_ids', 'hypothesis_attention_mask'])

In [ ]:
outputs_a = model(
    dataset[:1]['premise_input_ids'],
    attention_mask=dataset[:1]['premise_attention_mask']
)
outputs_a.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
outputs_a['pooler_output'].shape

torch.Size([1, 768])

In [ ]:
cos_sim = torch.nn.CosineSimilarity()

In [ ]:
outputs_b = model(
    dataset[:1]['hypothesis_input_ids'],
    attention_mask=dataset[:1]['hypothesis_attention_mask']
)
outputs_b.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
cos_sim(outputs_a.pooler_output, outputs_b.pooler_output)

tensor([0.9789], grad_fn=<DivBackward0>)

In [ ]:
# set device and move model there
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
print(f'moved to {device}')

moved to cuda


In [ ]:
ffnn = torch.nn.Linear(768*3, 3)
loss_func = torch.nn.CrossEntropyLoss()

ffnn.to(device)
loss_func.to(device)

CrossEntropyLoss()

In [ ]:
from transformers.optimization import get_linear_schedule_with_warmup

# we would initialize everything first
optim = torch.optim.Adam(model.parameters(), lr=1e-5)
# and setup a warmup for the first ~10% steps
total_steps = int(len(dataset) / batch_size)
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(
		optim, num_warmup_steps=warmup_steps,
  	num_training_steps=total_steps - warmup_steps
)

# then during the training loop we update the scheduler per step
scheduler.step()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [ ]:
from tqdm.auto import tqdm


TRAINING_MODE = False
epochs = 1

if TRAINING_MODE:
    for epoch in range(epochs):
        model.train()  # make sure model is in training mode
        # initialize the dataloader loop with tqdm (tqdm == progress bar)
        loop = tqdm(loader, leave=True)
        for batch in loop:
            # zero all gradients on each new step
            optim.zero_grad()
            
            # prepare batches and more all to the active device
            inputs_ids_a = batch['premise_input_ids'].to(device)
            inputs_ids_b = batch['hypothesis_input_ids'].to(device)
            attention_a = batch['premise_attention_mask'].to(device)
            attention_b = batch['hypothesis_attention_mask'].to(device)
            label = batch['label'].to(device)
            
            # extract token embeddings from BERT
            u = model(inputs_ids_a, attention_mask=attention_a)[0]  # all token embeddings A
            v = model(inputs_ids_b, attention_mask=attention_b)[0]  # all token embeddings B
            
            # get the mean pooled vectors
            u = mean_pool(u, attention_a)
            v = mean_pool(v, attention_b)
            
            # build the |u-v| tensor
            uv = torch.sub(u, v)
            uv_abs = torch.abs(uv)
            
            # concatenate u, v, |u-v|
            x = torch.cat([u, v, uv_abs], dim=-1)
            
            # process concatenated tensor through FFNN
            x = ffnn(x)
            
            # calculate the 'softmax-loss' between predicted and true label
            loss = loss_func(x, label)
            
            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()
            
            # update learning rate scheduler
            scheduler.step()
            # update the TDQM progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())

In [ ]:
import os

model_path = './output_nlp/sbert_test_a'

if not os.path.exists(model_path):
    os.mkdir(model_path)

if TRAINING_MODE:
  model.save_pretrained(model_path)
  tokenizer.save_pretrained(model_path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)
print(type(model))

<class 'transformers.models.bert.modeling_bert.BertModel'>


In [ ]:
output = "That is a happy person"
input_text = "That is a happy dog"
text2 = "That is a very happy person"
text3 = "I see an animal"

sentences = [output, input_text, text2, text3]

In [ ]:
import numpy as np

# build embeddings and calculate cosine similarity
def sts_process(sentence_a, sentence_b, model):
    vecs = []  # init list of sentence vecs
    for sentence in [sentence_a, sentence_b]:
        # build input_ids and attention_mask tensors with tokenizer
        input_ids = tokenizer(
            sentence, max_length=512, padding='max_length',
            truncation=True, return_tensors='pt'
        )
        # process tokens through model and extract token embeddings
        token_embeds = model(**input_ids).last_hidden_state
        # mean-pool token embeddings to create sentence embeddings
        sentence_embeds = mean_pool(token_embeds, input_ids['attention_mask'])
        vecs.append(sentence_embeds)
    # calculate cosine similarity between pairs and return numpy array
    return cos_sim(vecs[0], vecs[1]).detach().numpy()

In [ ]:
for i in range(1, len(sentences)):
  sim_score = sts_process(sentences[0], sentences[i], model)[0]
  print(f"Similarity between {sentences[0]} and {sentences[i]} is {sim_score}")

Similarity between That is a happy person and That is a happy dog is 0.6003449559211731
Similarity between That is a happy person and That is a very happy person is 0.9328435063362122
Similarity between That is a happy person and I see an animal is 0.19789300858974457
